In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [8]:
df = pd.read_csv("clean_data_label.csv")
df.shape
df.head(5)

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,CRS_ELAPSED_TIME,DISTANCE,DEP_DELAY,OP_UNIQUE_CARRIER_Label,ORIGIN_Label,DEST_Label
0,4,1,1,5.25,13.0,211,1477,11.0,0,2,21
1,4,1,1,13.32,16.0,223,1477,12.0,0,2,21
2,4,1,1,6.75,13.0,173,1020,-6.0,0,2,61
3,4,1,1,15.12,13.0,173,1020,-6.0,0,2,61
4,4,1,1,6.00,29.0,306,2279,-2.0,0,5,15


In [9]:
X = df.drop(columns=["DEP_DELAY"])
y = df["DEP_DELAY"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

Train size: (39334, 10), Test size: (9834, 10)


In [11]:
#--------------------------------------------------------#
# 3️⃣ Thiết lập tham số GridSearch
#--------------------------------------------------------#
param_grid = {
    'n_estimators': [100, 200, 300],       # Số cây
    'max_depth': [10, 20, None],           # Độ sâu
    'min_samples_split': [2, 5, 10],       # Số mẫu tối thiểu để tách node
    'min_samples_leaf': [1, 2, 4]          # Số mẫu tối thiểu ở node lá
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,                  # 3-fold CV
    scoring='r2',          # Đánh giá theo R²
    n_jobs=-1,             # Dùng toàn bộ CPU
    verbose=2
)


In [12]:
#--------------------------------------------------------#
# 4️⃣ Huấn luyện với GridSearchCV
#--------------------------------------------------------#
print("\n🔍 Đang tìm tham số tốt nhất qua cross-validation (cv=3)...")
grid.fit(X_train, y_train)

print("\n✅ Grid Search hoàn tất!")
print("Best Parameters:", grid.best_params_)
print(f"Best CV R² Score: {grid.best_score_:.4f}")


🔍 Đang tìm tham số tốt nhất qua cross-validation (cv=3)...
Fitting 3 folds for each of 81 candidates, totalling 243 fits

✅ Grid Search hoàn tất!
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Best CV R² Score: 0.0248


In [13]:
#--------------------------------------------------------#
# 5️⃣ Đánh giá trên test set
#--------------------------------------------------------#
best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2   = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test.replace(0, np.nan))) * 100

print("\n📊 Hiệu năng Random Forest (với best parameters):")
print(f"MAE   = {mae:.3f} phút")
print(f"RMSE  = {rmse:.3f} phút")
print(f"R²    = {r2:.4f}")
print(f"MAPE  = {mape:.2f}%")


📊 Hiệu năng Random Forest (với best parameters):
MAE   = 17.531 phút
RMSE  = 35.031 phút
R²    = 0.0184
MAPE  = 272.89%
